In [8]:
import numpy as np
import json
import os
import zipfile
import wget
import csv
import socket
import subprocess


In [2]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir) if os.path.isdir(os.path.join(a_dir, name))]

def get_immediate_files(a_dir):
    return [name for name in os.listdir(a_dir) if os.path.isfile(os.path.join(a_dir, name))]

In [3]:
label_map = {}
with open('./data/class_labels_indices.csv') as csv_file:
    
# label_set = np.loadtxt('./data/class_labels_indices.csv', delimiter=',', dtype='str')
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
#             print(f'\t{row[0]}{row[1]} {row[2]}.')
            label_map[row[2]] = row[0]
            line_count += 1
#     label_set = list(label_set)
#     for i in range(1, len(label_set)):
#         label_map[eval(label_set[i][2])] = label_set[i][0]
print(label_map)

Column names are index, mid, display_name
{'Speech': '0', 'Male speech, man speaking': '1', 'Female speech, woman speaking': '2', 'Child speech, kid speaking': '3', 'Conversation': '4', 'Narration, monologue': '5', 'Babbling': '6', 'Speech synthesizer': '7', 'Shout': '8', 'Bellow': '9', 'Whoop': '10', 'Yell': '11', 'Battle cry': '12', 'Children shouting': '13', 'Screaming': '14', 'Whispering': '15', 'Laughter': '16', 'Baby laughter': '17', 'Giggle': '18', 'Snicker': '19', 'Belly laugh': '20', 'Chuckle, chortle': '21', 'Crying, sobbing': '22', 'Baby cry, infant cry': '23', 'Whimper': '24', 'Wail, moan': '25', 'Sigh': '26', 'Singing': '27', 'Choir': '28', 'Yodeling': '29', 'Chant': '30', 'Mantra': '31', 'Male singing': '32', 'Female singing': '33', 'Child singing': '34', 'Synthetic singing': '35', 'Rapping': '36', 'Humming': '37', 'Groan': '38', 'Grunt': '39', 'Whistling': '40', 'Breathing': '41', 'Wheeze': '42', 'Snoring': '43', 'Gasp': '44', 'Pant': '45', 'Snort': '46', 'Cough': '47', 

In [4]:
# fix bug: generate an empty directory to save json files
if os.path.exists('./data/datafiles') == False:
    os.mkdir('./data/datafiles')

In [9]:
curr_node = socket.gethostname().split('.')[0]
batcmd=f"squeue -u billyli | grep {curr_node}"
curr_slurm = subprocess.check_output(batcmd, shell=True, text=True)
slurm_id = curr_slurm.strip().split(' ')[0]
print(slurm_id)

12056603


In [10]:
list_path = "./lists/"
base_path = f"/local/slurm-{slurm_id}/local/audio"
# meta = np.loadtxt('./data/ESC-50-master/meta/esc50.csv', delimiter=',', dtype='str', skiprows=1)

bal_train_wav_list = []
unbal_train_wav_list = []
eval_wav_list = []
bal_unbal_train_wav_list = []
with open(os.path.join(list_path, 'unbalanced_train_segments.csv')) as unbal_csv_file:
    unbal_csv_reader = csv.reader(unbal_csv_file, delimiter=',')

    unbal_count = 0
    missing_count = 0
    for row in unbal_csv_reader:
        if unbal_count <3:
            print(f'Column names are {", ".join(row)}')
            unbal_count += 1
        else:
            wav_path = os.path.join(base_path, 'unbalanced_wav',row[0] +'.wav')
            if os.path.exists(wav_path):
                cur_unbal_dict = {"wav": wav_path, "labels": eval(','.join(row[3:]))}
#             print(cur_unbal_dict)
                unbal_train_wav_list.append(cur_unbal_dict)
                bal_unbal_train_wav_list.append(cur_unbal_dict)
                unbal_count += 1

            else:
                missing_count+=1
#                 print("missing training: " + row[0])
print(f'unbalanced count: {unbal_count}, missing count: {missing_count}')
with open('./data/datafiles/audioset_unbal_train_data' +'.json', 'w') as f:
    json.dump({'data': unbal_train_wav_list}, f, indent=1)

with open(os.path.join(list_path, 'balanced_train_segments.csv')) as bal_csv_file:
    bal_csv_reader = csv.reader(bal_csv_file, delimiter=',')

    bal_count = 0
    missing_count = 0
    for row in bal_csv_reader:
        if bal_count <3:
            print(f'Column names are {", ".join(row)}')
            bal_count += 1
        else:
            wav_path = os.path.join(base_path, 'balance_wav',row[0] +'.wav')
            if os.path.exists(wav_path):
                cur_bal_dict = {"wav": wav_path, "labels": eval(','.join(row[3:]))}
#             print(cur_bal_dict)
                bal_train_wav_list.append(cur_bal_dict)
                bal_unbal_train_wav_list.append(cur_bal_dict)
                bal_count += 1

            else:
                missing_count+=1
#                 print("missing training: " + row[0])
print(f'balanced count: {bal_count}, missing count: {missing_count}')
with open('./data/datafiles/audioset_bal_train_data' +'.json', 'w') as f:
    json.dump({'data': bal_train_wav_list}, f, indent=1)

with open('./data/datafiles/audioset_bal_unbal_train_data' +'.json', 'w') as f:
    json.dump({'data': bal_unbal_train_wav_list}, f, indent=1)
    
with open(os.path.join(list_path, 'eval_segments.csv')) as eval_csv_file:
    eval_csv_reader = csv.reader(eval_csv_file, delimiter=',')

    eval_count = 0
    missing_count = 0
    for row in eval_csv_reader:
        if eval_count <3:
            print(f'Column names are {", ".join(row)}')
            eval_count += 1
        else:
            wav_path = os.path.join(base_path, 'eval_wav',row[0] +'.wav')
            if os.path.exists(wav_path):
                cur_eval_dict = {"wav": wav_path, "labels": eval(','.join(row[3:]))}
#             print(cur_bal_dict)
                eval_wav_list.append(cur_eval_dict)
                eval_count += 1

            else:
                missing_count+=1
#                 print("missing training: " + row[0])
print(f'eval count: {eval_count}, missing count: {missing_count}')
with open('./data/datafiles/audioset_eval_data' +'.json', 'w') as f:
    json.dump({'data': eval_wav_list}, f, indent=1)
    
# for i in range(0, len(meta)):
#     cur_label = label_map[meta[i][3]]
#     cur_path = meta[i][0]
#     cur_fold = int(meta[i][1])
#     # /m/07rwj is just a dummy prefix
#     cur_dict = {"wav": base_path + cur_path, "labels": '/m/07rwj'+cur_label.zfill(2)}
#     if cur_fold == fold:
#         eval_wav_list.append(cur_dict)
#     else:
#         train_wav_list.append(cur_dict)

# print('fold {:d}: {:d} training samples, {:d} test samples'.format(fold, len(train_wav_list), len(eval_wav_list)))

# with open('./data/datafiles/esc_train_data_'+ str(fold) +'.json', 'w') as f:
#     json.dump({'data': train_wav_list}, f, indent=1)

# with open('./data/datafiles/esc_eval_data_'+ str(fold) +'.json', 'w') as f:
#     json.dump({'data': eval_wav_list}, f, indent=1)

print('Finished AudioSet Preparation')

Column names are # Segments csv created Sun Mar  5 10:56:58 2017
Column names are # num_ytids=2041789,  num_segs=2041789,  num_unique_labels=527,  num_positive_labels=4020212
Column names are # YTID,  start_seconds,  end_seconds,  positive_labels
unbalanced count: 1998999, missing count: 42793
Column names are # Segments csv created Sun Mar  5 10:54:31 2017
Column names are # num_ytids=22160,  num_segs=22160,  num_unique_labels=527,  num_positive_labels=52882
Column names are # YTID,  start_seconds,  end_seconds,  positive_labels
balanced count: 21720, missing count: 443
Column names are # Segments csv created Sun Mar  5 10:54:25 2017
Column names are # num_ytids=20371,  num_segs=20371,  num_unique_labels=527,  num_positive_labels=51804
Column names are # YTID,  start_seconds,  end_seconds,  positive_labels
eval count: 20126, missing count: 248
Finished AudioSet Preparation


In [11]:
os.path.exists(f'/local/slurm-{slurm_id}/local/audio/balance_wav/zy8mUJijw3o.wav')

True